In [31]:
import cv2 as cv
import mediapipe as mp
import time
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import pathlib

In [32]:
segments = []
recording = []

In [33]:
def segmentize(recording_arr):
    segments.append(recording_arr[0]) # top_1
    
    prev_frame_yval = round(recording_arr[0][1], 1) # NOTE: The 2 index here is he 2nd decimal place
    change_idx = 0
    
    next_frame_yval = round(recording_arr[len(recording_arr)-2][1], 1)
    change_idx_bot = 0
    
    for i in range(len(recording_arr)):
        if round(recording_arr[i][1], 1) > prev_frame_yval: # This is where the change starts from top_1 
            change_idx = i
            break
            
    for j in range(len(recording_arr)):
        if round(recording_arr[j][1], 1) == next_frame_yval: # This is where the change starts to bot_1
            change_idx_bot = j
            break
            
    segments.append(recording_arr[math.floor((change_idx_bot + change_idx)/2)]) # mid_1
    segments.append(recording_arr[len(recording_arr)-2]) # bot_1
    
    print("Segmentized!")
    return True

In [34]:
def save_imgs():
    counter = 0
    for i in segments:
        cv.imwrite(f"{os.path.abspath(os.getcwd())}\\cnn_inputs\\input{counter}.png", i[0])
        counter += 1
    
    
    print("Images saved!")
    recording.clear()
    segments.clear()

In [35]:
# pose estimation drawing utilities
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

# One time camera snapshot
camera = cv.VideoCapture(0, cv.CAP_DSHOW)
_, cam_frame = camera.read()
camera.release()
cv.destroyAllWindows()
cam_shape = cam_frame.shape

# Recording ulitilites
prev_img = np.zeros(cam_shape)

# timer utilities
initial_time = time.time()
initial_ankle_pos = 0.01
isTimed = False
isRecording = False
camera = cv.VideoCapture(0, cv.CAP_DSHOW)
tmp_lmval =  0.1

monitor_val = 0.01
isTracking = False
rec_this = 0
while True:
    ret, frame = camera.read()

    imgRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    
    if results.pose_landmarks:
        mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = frame.shape
    
            if id == 11: 
                tmp_lmval = round(lm.x, 1)
                
                
            if (not isTimed): # Set timer for 2 seconds
                initial_time = time.time()
                if id == 11: 
                    initial_ankle_pos = round(lm.x, 1)
            else: 
                if time.time() >= initial_time + 2:
                    isRecording = True
                
                
            if tmp_lmval == initial_ankle_pos: 
                isTimed = True
            else:
                isTimed = False
            
            if isRecording:
                if id == 11:
                    recording.append((frame, round(lm.y, 1)))
                    #monitor_val = round(lm.x, 1)
                    rec_this = round(lm.y, 1)
                    
                    if monitor_val > rec_this:
                        isRecording = False
                        isTracking = segmentize(recording) 
                        monitor_val = 0.01
                        isTimed = False
                        initial_time = time.time()
                        initial_time2 = time.time()
                        
                    monitor_val = round(lm.y, 1)
            
            if isTracking:
                if id == 11:
                
                    if monitor_val < round(lm.y, 1) or recording[0][1] == round(lm.y, 1):
                        isTracking = False
                        recording.append((frame, round(lm.y, 1)))
                        segments.append(recording[len(recording)-1])
                        save_imgs()
                        isRecording = False
                        isTimed = False
                        initial_time = time.time()
                        initial_time2 = time.time()
                        monitor_val = 0.01
                        print("Squat Ended!")
                
                
          
        
    cv.imshow("Realtime", cv.putText(frame, f"{isRecording}-{monitor_val}", (40, 50), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2))
    key = cv.waitKey(1)
    
    if key == ord('q'):
        key = cv.waitKey()
        camera.release()
        cv.destroyAllWindows()
        break
        

Segmentized!
Images saved!
Squat Ended!
